In [163]:
# 1. import libraries
from bs4 import BeautifulSoup
import requests
import pandas as pd
df = pd.read_csv("billboard100_data.csv")

In [164]:
df['title'] = df['title'].map(str.lower) # converting the entries to lower
df['author'] = df['author'].map(str.lower) # converting the entries to lower
df

,title,author
0,flowers,miley cyrus
1,kill bill,sza
2,last night,morgan wallen
3,unholy,sam smith & kim petras
4,creepin',"metro boomin, the weeknd & 21 savage"
...,...,...
95,yandel 150,yandel & feid
96,heart to heart,mac demarco
97,next thing you know,jordan davis
98,never gonna not dance again,p!nk


In [166]:
x = input('Enter a song you like: ')
if x.lower() in df.values:   # converts te entered x into all lower case
        rec_song = df['title'].sample().to_string(index=False) # recommendation song. I just removed the indexes as they are not required as the output  
        rec_artist = df.loc[df['title'] == rec_song, 'author'].to_string(index=False) # artist of the recommended song
        # to_string(index=False) lets you output without index
        if x != rec_song:
            print("You might like this song: " + rec_song.title() + " by " + rec_artist.title())  # .title() makes the first alphabet in every word to upper   
else:
        print("\nThis song does not exists in the data set.")
        

Enter a song you like: next thing you know
You might like this song: Amg by Gabito Ballesteros, Peso Pluma & Natanael Cano
